In [ ]:
import os
import sys
import glob
import copy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score
# import ipywidgets as widgets
# import bqplot.pyplot as bqplt
# from tqdm.notebook import tqdm
from IPython.core.interactiveshell import InteractiveShell
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
# import tensorflow as tf
# ['all', 'last', 'last_expr', 'none', 'last_expr_or_assign']
InteractiveShell.ast_node_interactivity = "last"
# matplotlib configuration
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5
mpl.rcParams['font.size'] = 12
# plt.style.use(['dark_background'])
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


## Load DSA dataset

In [ ]:
# %run ../src/mex_loader.py

features = ['act1', 'act2', 'act3', 'acw1', 'acw2', 'acw3']


class MEXDataset:
    def __init__(self, data_path, exercises=None, train_rate=0.5, nb_views=2):
        self.data_path = data_path
        self.nb_views = nb_views
        self.features = features
        self.train_rate = train_rate
        self.exercises = exercises
        self.views = ['act', 'acw']

    def load_data(self):
        train_views_dfs, test_views_dfs = {}, {}
        min_length = 10000000
        tmp_views_dfs = {}
        for v, view in enumerate(self.views):
            print(f'Loading {view}...')
            p_paths = sorted(glob.glob(f"{self.data_path}/{view}/*"))
            features = ['time'] + [feat for feat in self.features if view in feat]
            tmp_views_dfs[f'view_{v+1}'] = {}
            for p_path in p_paths:
                p_key = p_path.split('/')[-1]
                e_paths = [f"{p_path}/{ex}_{view}_1.csv" for ex in self.exercises]
                dfs = {
                    f"e{ex}_p{p_key}": pd.read_csv(path, names=features).drop(columns=['time']) 
                    for ex, path in zip(self.exercises, e_paths)
                    }
                min_length = min(min_length, *[df.shape[0] for df in dfs.values()])
                for key, df in dfs.items():
                    tmp_views_dfs[f'view_{v+1}'][key] = df
        train_split = int(min_length*self.train_rate)
        for view, view_dfs in tmp_views_dfs.items():
            train_views_dfs[view], test_views_dfs[view] = {}, {}
            for ep, df in view_dfs.items():
                train_views_dfs[view][ep] = df[:min_length]
                test_views_dfs[view][ep] = df[:min_length].reset_index().drop(columns=['index'])
        return train_views_dfs, test_views_dfs
    
    def save_into_features(self, stored_dir, train_views_dfs):
        print("Saving features into files")
        for view, view_dfs in train_views_dfs.items():
            view_path = stored_dir+f"/raw/{view}"
            if not os.path.exists(view_path):
                os.makedirs(view_path)
            for ap, df in view_dfs.items():
                for col in df.columns:
                    path = f"{view_path}/{col}"
                    os.makedirs(path, exist_ok=True)
                    df[col].to_csv(f"{path}/{ap}.csv", header=[col])


exercises = ['01', '02', '03', '05', '06', '07']
mex_dataset = MEXDataset("../raw_datasets/mex", exercises=exercises)
train_views_dfs, test_views_dfs = mex_dataset.load_data()
stored_dir = '../preprocessed_datasets/mex'
mex_dataset.save_into_features(stored_dir, train_views_dfs)

In [ ]:
import json

def get_scores(view, feature, instances):
    score_path = f"/home/tuangroup/ntphuong/projects/MVAD/preprocessed_datasets/mex/gen_data/{view}/{feature}_saved_scores.json"
    with open(score_path, 'r') as f:
        saved_scores = json.load(f)
    anomaly_scores_np = np.array([saved_score[1] for saved_score in saved_scores])
    print(anomaly_scores_np.shape)
    anomaly_scores = pd.DataFrame(anomaly_scores_np.T, columns=[f'{i}' for i in instances])
    return anomaly_scores

def plot_instance(instance, view, feature, instances):
    anomaly_score = get_scores(view, feature, instances)
    fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(20, 7*2))
    data = train_views_dfs[view][instance][feature]
    axs[0].plot(data)
    predict_indices = anomaly_score[instance].sort_values(ascending=False)[:100].index
    axs[0].scatter(predict_indices, data[predict_indices], c='r')
    # new_data = data.copy(deep=True)
    # for idx in predict_indices:
    #     new_data[idx] = 0.
    # axs[1].plot(new_data)
    axs[1].plot(anomaly_score[instance])
    plt.show()

def replace_anomaly_values(view, feature, instances):
    anomaly_scores = get_scores(view, feature, instances)
    for instance in instances:
        predict_indices = anomaly_scores[instance].sort_values(ascending=False)[:20].index
        for idx in predict_indices:
            train_views_dfs[view][instance][feature][idx] = np.nan
            test_views_dfs[view][instance][feature][idx] = np.nan
        # train_views_dfs[view][instance].fillna(method='ffill', inplace=True)
        # test_views_dfs[view][instance].fillna(method='ffill', inplace=True)
        # NOTE: We use linear method since it is the only method that supports MultiIndexes
        train_views_dfs[view][instance].interpolate(method='linear', inplace=True)
        test_views_dfs[view][instance].interpolate(method='linear', inplace=True)
        


# instance = 'a01_p10'
# view = 'view_2'
# feature = 'mag_lankle_y'
# instances = sorted(list(train_views_dfs['view_1'].keys()))
# plot_instance(instance, view, feature, instances)
# replace_anomaly_values(view, feature, instances)
# plot_instance(instance, view, feature, instances)
for view, view_dfs in train_views_dfs.items():
    instances = sorted(list(train_views_dfs[view].keys()))
    for feature in view_dfs[instances[0]].columns:
        replace_anomaly_values(view, feature, instances)

In [ ]:
for view in train_views_dfs.keys():
    for instance in train_views_dfs['view_1'].keys():
        split_point = train_views_dfs[view][instance].shape[0] // 2
        train_views_dfs[view][instance] = pd.DataFrame(train_views_dfs[view][instance][:split_point].values, columns=train_views_dfs[view][instance].columns)
        test_views_dfs[view][instance] = pd.DataFrame(test_views_dfs[view][instance][split_point:].values, columns=test_views_dfs[view][instance].columns)
print(train_views_dfs['view_1']['e01_p01'].shape)
print(test_views_dfs['view_1']['e01_p01'].shape)

## Generate Anomalies

In [ ]:
%run ../src/anomaly_generations.py

dataset_name = 'mex_timestep_same_subject_random_view'
n_samples = 15
nb_views = 2

for sample in range(6, n_samples+1):
    for anomaly_rate in [5, 10, 15, 20]:
        clusters = [f'e{ex}' for ex in exercises]
        dir_path = f"../preprocessed_datasets/{dataset_name}/sample{sample}/anomaly_rate_{anomaly_rate}_views_{nb_views}"
        swapped_test_views_dfs, ground_truths = swap_time_steps(copy.deepcopy(
            test_views_dfs), clusters=clusters, anomaly_rate=anomaly_rate*0.01)

        # Save to files
        print("Saving files...")
        for view, view_dfs in train_views_dfs.items():
            view_path = dir_path+f"/train/{view}"
            if not os.path.exists(view_path):
                os.makedirs(view_path)
            for ap, df in view_dfs.items():
                df.to_csv(f"{view_path}/{ap}.csv", index=False)
        for view, view_dfs in swapped_test_views_dfs.items():
            view_path = dir_path+f"/test/{view}"
            if not os.path.exists(view_path):
                os.makedirs(view_path)
            for ap, df in view_dfs.items():
                df.to_csv(f"{view_path}/{ap}.csv", index=False)
        for ap, gt in ground_truths.items():
            gt.to_csv(dir_path+f"/test/{ap}.csv", index=False)
        print('Done.')
